# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Описание данных

Данные геологоразведки трёх регионов .   
id — уникальный идентификатор скважины;  
f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);  
product — объём запасов в скважине (тыс. баррелей).  
**Условия задачи:**  
При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.  
Бюджет на разработку скважин в регионе — 10 млрд рублей.  
При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.  
После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.  
Данные синтетические: детали контрактов и характеристики месторождений не разглашаются.

## Загрузка и подготовка данных

In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from numpy.random import RandomState
from scipy import stats as st

from sklearn.linear_model import LinearRegression

In [2]:
RANDOM = 12345
STATE = RandomState(RANDOM)

In [3]:
# константы расчетов 
# количество точек для исследования
POINTS_RESEARCH = 500
# количество точек лучших для разработки
BEST_DEVELOPMENT = 200 
# бюджет на разработку скважин в регионе в млн. руб.
BUDGET = 10000
# доход с 1 тыс баррель в млн. руб
INCOME_BARREL = 0.45
# максимальная вероятность убытков 2.5 %
LOSSES_PROBABILITY = 0.025

In [4]:
try:
    geo_data_0 = pd.read_csv('/datasets/geo_data_0.csv')
    geo_data_1 = pd.read_csv('/datasets/geo_data_1.csv')
    geo_data_2 = pd.read_csv('/datasets/geo_data_2.csv')
except:
    geo_data_0 = pd.read_csv('C:\\Users\\Константин\\Desktop\\Файлы\\Модуль 2\\geo_data_0.csv')
    geo_data_1 = pd.read_csv('C:\\Users\\Константин\\Desktop\\Файлы\\Модуль 2\\geo_data_1.csv')
    geo_data_2 = pd.read_csv('C:\\Users\\Константин\\Desktop\\Файлы\\Модуль 2\\geo_data_2.csv')

In [5]:
display(geo_data_0.head())
print()
print(geo_data_0.info())
print()
display(geo_data_0.describe())
print()
print('Дубликаты:', geo_data_0.duplicated().sum())

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None



,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347



Дубликаты: 0


In [6]:
display(geo_data_1.head())
print()
print(geo_data_1.info())
print()
display(geo_data_1.describe())
print()
print('Дубликаты:', geo_data_1.duplicated().sum())

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None



,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408



Дубликаты: 0


In [7]:
display(geo_data_2.head())
print()
print(geo_data_2.info())
print()
display(geo_data_2.describe())
print()
print('Дубликаты:', geo_data_2.duplicated().sum())

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None



,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838



Дубликаты: 0


Данные не имеют пропусков и дубликатов, нормально распределены, типы данных корректны. Единственное, что требуется, удалить столбец с наименованиями, что бы он не мешал обучению.

In [8]:
geo_data_0 = geo_data_0.drop('id', axis=1)
geo_data_1 = geo_data_1.drop('id', axis=1)
geo_data_2 = geo_data_2.drop('id', axis=1)

In [9]:
geo_data_0.head()

,f0,f1,f2,product
0,0.705745,-0.497823,1.221170,105.280062
1,1.334711,-0.340164,4.365080,73.037750
2,1.022732,0.151990,1.419926,85.265647
3,-0.032172,0.139033,2.978566,168.620776
4,1.988431,0.155413,4.751769,154.036647


## Обучение и проверка модели

Разделим данные на обучающую и валидационную выборку. Обучим модель линейной регрессия (остальные — недостаточно предсказуемые).

In [10]:
# функцию разделения данных на обучающую и валидационную выборки
def division (geo_data):
    features = geo_data.drop('product', axis=1)
    target = geo_data['product']
    
    # выборки в соотношении 75:25
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, 
                                                                                  test_size=0.25, 
                                                                                  random_state=RANDOM)
    # масштабируем количественные признаки
    numeric = ['f0', 'f1', 'f2']
    scaler = StandardScaler()
    scaler.fit(features_train[numeric])

    features_train[numeric] = scaler.transform(features_train[numeric])
    features_valid[numeric] = scaler.transform(features_valid[numeric])
    
    return features_train, features_valid, target_train, target_valid 

In [11]:
# создадим обучающую и валидационную выборки для 3-х регионов
features_train_0, features_valid_0, target_train_0, target_valid_0 = division(geo_data_0)
features_train_1, features_valid_1, target_train_1, target_valid_1 = division(geo_data_1)
features_train_2, features_valid_2, target_train_2, target_valid_2 = division(geo_data_2)

# проверим разделение 
print(features_train_0.shape)
print(features_valid_0.shape)
print(target_train_0.shape)
print(target_valid_0.shape)
print()
print(features_train_1.shape)
print(features_valid_1.shape)
print(target_train_1.shape)
print(target_valid_1.shape)
print()
print(features_train_2.shape)
print(features_valid_2.shape)
print(target_train_2.shape)
print(target_valid_2.shape)

# проверим применение масштабирования
display(features_train_0.describe())
display(features_train_1.describe())
features_train_2.describe()

(75000, 3)
(25000, 3)
(75000,)
(25000,)

(75000, 3)
(25000, 3)
(75000,)
(25000,)

(75000, 3)
(25000, 3)
(75000,)
(25000,)


,f0,f1,f2
count,7.500000e+04,7.500000e+04,7.500000e+04
mean,1.680907e-16,-3.474554e-17,6.442254e-18
std,1.000007e+00,1.000007e+00,1.000007e+00
min,-2.186288e+00,-2.178269e+00,-3.890945e+00
25%,-6.575544e-01,-8.925344e-01,-6.799576e-01
50%,1.878842e-03,-7.407961e-04,4.288679e-03
75%,6.571176e-01,8.928514e-01,6.829752e-01
max,2.139081e+00,2.169192e+00,4.153623e+00


,f0,f1,f2
count,7.500000e+04,7.500000e+04,7.500000e+04
mean,-1.136868e-18,-2.717115e-16,-2.030968e-17
std,1.000007e+00,1.000007e+00,1.000007e+00
min,-3.652789e+00,-4.206546e+00,-1.476535e+00
25%,-8.290577e-01,-6.766541e-01,-8.784844e-01
50%,1.734177e-03,-1.083306e-03,-2.838957e-01
75%,8.349682e-01,6.768381e-01,8.834721e-01
max,3.154487e+00,4.588691e+00,1.482457e+00


,f0,f1,f2
count,7.500000e+04,7.500000e+04,7.500000e+04
mean,-2.122154e-17,7.294905e-18,-8.251770e-17
std,1.000007e+00,1.000007e+00,1.000007e+00
min,-5.060306e+00,-4.088636e+00,-4.167323e+00
25%,-6.716773e-01,-6.783003e-01,-6.785573e-01
50%,3.845213e-03,-5.219873e-03,-5.092214e-03
75%,6.656278e-01,6.749474e-01,6.802621e-01
max,4.176529e+00,4.533914e+00,4.101780e+00


In [12]:
# создадим таблицу для хранения результатов вычисления RMSE и сравнения предсказания с истинным значением
table_1 = pd.DataFrame({'RMSE': [], 'Среднее предсказание': [],'Среднее истинное': []} )

In [13]:
# создаем функцию для получения предсказаний на модели LinearRegression()
def training (features_train, features_valid, target_train, target_valid):
    
    # создадим и обучим модель 
    model = LinearRegression() 
    model.fit(features_train, target_train)
    
    # получим предсказания 
    predictions_valid = pd.Series(model.predict(features_valid)) 
    
    # считаем значение метрики RMSE 
    rmse = mean_squared_error(target_valid, predictions_valid)**0.5
    
    # расчитываем средний запас сырья 
    mean_stock = predictions_valid.mean()
    
    # внесем данные в таблицу
    table_1.loc[ len(table_1.index )] = [rmse, mean_stock, target_valid.mean()]
    
    return predictions_valid, rmse, mean_stock
    

In [14]:
predictions_valid_0, rmse_0, mean_stock_0 = training(features_train_0, features_valid_0, target_train_0, target_valid_0)
predictions_valid_1, rmse_1, mean_stock_1 = training(features_train_1, features_valid_1, target_train_1, target_valid_1)
predictions_valid_2, rmse_2, mean_stock_2 = training(features_train_2, features_valid_2, target_train_2, target_valid_2)

In [15]:
# добавим столбец с регионами
Name = pd.Series({'Регион': ['№0 ','№1 ','№2 ' ]})
table_1['Регион'] = Name['Регион']
table_1

,RMSE,Среднее предсказание,Среднее истинное,Регион
0,37.579422,92.592568,92.078597,№0
1,0.893099,68.728547,68.723136,№1
2,40.029709,94.965046,94.884233,№2


Предсказания по всем 3 регионам близкие к действительным. Самая большая точность была установлена по данным региона №1. Регионы №0 и №2 имею практически одинаковую погрешность в измерениях. 

## Подготовка к расчёту прибыли

Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины. Сравним полученный объём сырья со средним запасом в каждом регионе.

In [16]:
# затраты на разработку одной скважины.
expenses = BUDGET / BEST_DEVELOPMENT
print('Допустимые затраты на разработку одной скважины составляют', expenses, 'млн. руб.')

# количество в тыс. баррель необходимо для покрытия затрат 
sufficient_volume = expenses / INCOME_BARREL
print('Необходимый объем сырья для безубыточной разработки 1 скважины' ,'{:.2f}'.format(sufficient_volume), 'тыс. баррелей')

# среднее количество в тыс. баррель по регионам
print('Средний объем сырья региона №0 в 1 скавжине','{:.2f}'.format(geo_data_0['product'].mean()), 'тыс. баррелей ')
print('Средний объем сырья региона №1 в 1 скавжине', '{:.2f}'.format(geo_data_1['product'].mean()), 'тыс. баррелей ')
print('Средний объем сырья региона №2 в 1 скавжине', '{:.2f}'.format(geo_data_2['product'].mean()), 'тыс. баррелей ')

Допустимые затраты на разработку одной скважины составляют 50.0 млн. руб.
Необходимый объем сырья для безубыточной разработки 1 скважины 111.11 тыс. баррелей
Средний объем сырья региона №0 в 1 скавжине 92.50 тыс. баррелей 
Средний объем сырья региона №1 в 1 скавжине 68.83 тыс. баррелей 
Средний объем сырья региона №2 в 1 скавжине 95.00 тыс. баррелей 


Сравнение среднего показателя по скважинам в регионе с объёмом добываемого сырья необходимого, что бы скважина не приносила убытков показывает, что выбор скважин случайным образом рискованное предприятие. 

Посчитаем прибыль по 200 лучшим предсказаниям сделанными моделью.

In [17]:
# создаем функцию для расчета максимально возможной прибыли по предсказаниям модели
def best_profit (target_valid, predictions_valid):
    
    # выбирем 200 скважин с максимальным объемом нефти
    pred_max = predictions_valid.sort_values(ascending=False).head(200) 
    target_max = target_valid.reset_index(drop=True)[pred_max.index] 
    
    # определим прибль 
    income = (target_max.sum() * INCOME_BARREL) - BUDGET
    
    return income

In [18]:
print('Прибыль с 200 лучших скважин предсказанных моделью составит:')
print('для региона №0 - {:.2f} млн. руб.'.format(best_profit(target_valid_0, predictions_valid_0)))
print('для региона №1 - {:.2f} млн. руб.'.format(best_profit(target_valid_1, predictions_valid_1)))
print('для региона №2 - {:.2f} млн. руб.'.format(best_profit(target_valid_2, predictions_valid_2)))

Прибыль с 200 лучших скважин предсказанных моделью составит:
для региона №0 - 3320.83 млн. руб.
для региона №1 - 2415.09 млн. руб.
для региона №2 - 2710.35 млн. руб.


Если исследовать все скважины в регионе и сделать по ним предсказания, то минимальная прибыль составит почти четверть от вложенных средств. Выделенный бюджет позволяет исследовать только 500 скважен. Подсчитаем риски с этим связанные. 

## Расчёт прибыли и рисков 

In [19]:
# создадим таблицу для хранения результатов вычисления прибыли и рисков
table_2 = pd.DataFrame({'95%-ый интервал от': [],
                        '95%-ый интервал до': [],
                        '2.5%-квантиль': [],
                        'Вероятность убытков': [],
                        'Средняя прибыль': []} )

In [20]:
def profit (target_valid, predictions_valid, best_count):
    # сортируем скважины по значениям их предсказаний
    pred_max = predictions_valid.sort_values(ascending=False) 
    
    # определяем верные значения, выбирем 200 скважин с максимальным объемом нефти
    target_max = target_valid[pred_max.index][:best_count]
    
    # определим прибль
    income = (target_max.sum() * INCOME_BARREL) - BUDGET
    return income

In [21]:
# создадим функцию подсчета рисков для каждого региона
def risks_and_profits (predictions_valid, target_valid, count):
    
    # cчетчик убыточных скважен
    counte = 0
    
    # Переменная, в которой будут храниться все значения выручек для 1000 выборок
    values = [] 
    
    # создаем 1000 выборок
    for i in range(1000): 
        target_subsample = target_valid.reset_index(drop=True).sample(count, replace=True, random_state=STATE)
        probs_subsample = predictions_valid[target_subsample.index]
        
        # считаем прибыль следуемых точек, по 200 лучшим предсказаниям
        profits = profit(target_subsample, probs_subsample, BEST_DEVELOPMENT) 
        values.append(profits)
        
        if profits < 0:
            counte += 1
        
    values = pd.Series(values)
    
    #посчитаем и выведем среднюю прибыль
    mean = values.mean()
    
    # посчитаем % убытков
    losses = counte / 1000
    
    # определим 2.5 %-квантиль
    lower = values.quantile(LOSSES_PROBABILITY)
    
    # посчитаем 95-% доверительный интервал
    confidence_interval = st.t.interval(0.95, len(values)-1, values.mean(), values.sem())
   
    table_2.loc[ len(table_2.index )] = ['{:.2f} млн.'.format(confidence_interval[0]),
                                         '{:.2f} млн.'.format(confidence_interval[1]),
                                         '{:.2f} млн.'.format(lower),
                                         '{:.2%}'.format(losses),
                                         '{:.2f} млн.'.format(mean)]

In [22]:
# отбираем 500 скважин для исследования
risks_and_profits(predictions_valid_0, target_valid_0, POINTS_RESEARCH)
risks_and_profits(predictions_valid_1, target_valid_1, POINTS_RESEARCH)
risks_and_profits(predictions_valid_2, target_valid_2, POINTS_RESEARCH)

In [23]:
# добавляем в таблицу название региона и максимально возможную прибыль
Name = pd.Series({'Регион': ['№0 ','№1 ','№2 ' ]})
max_p = pd.Series({'Масимальная средняя прибыль': ['3320.82 млн.','2415.09 млн.','2710.35 млн.' ]})
table_2['Масимальная средняя прибыль'] = max_p['Масимальная средняя прибыль']
table_2['Регион'] = Name['Регион']
# сортируем по % убытков 
table_2.sort_values('Вероятность убытков',ascending=True)

,95%-ый интервал от,95%-ый интервал до,2.5%-квантиль,Вероятность убытков,Средняя прибыль,Масимальная средняя прибыль,Регион
1,505.25 млн.,531.27 млн.,128.12 млн.,0.30%,518.26 млн.,2415.09 млн.,№1
0,408.73 млн.,443.14 млн.,-102.09 млн.,6.00%,425.94 млн.,3320.82 млн.,№0
2,402.53 млн.,437.86 млн.,-115.85 млн.,6.20%,420.19 млн.,2710.35 млн.,№2


# Вывод

Исследования 500 точек, 200 из которых были отобраны с помощью прогнозов машинного обучения показывают. Заданному условию, что вероятность убытков составит менее 2.5% соответствует регион №1. Так же регион №1 имеет самый высоки показатель средней прибыли равный 517.34 млн. Предлогаю выбрать это регион для разработки месторождений.